In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unzip '/content/drive/MyDrive/Colab Notebooks/Car damage-20221109T043527Z-001.zip'

Archive:  /content/drive/MyDrive/Colab Notebooks/Car damage-20221109T043527Z-001.zip
  inflating: Car damage/level/validation/01-minor/0038.JPEG  
  inflating: Car damage/level/validation/01-minor/0033.JPEG  
  inflating: Car damage/level/validation/01-minor/0031.JPEG  
  inflating: Car damage/level/validation/01-minor/0020.JPEG  
  inflating: Car damage/level/validation/01-minor/0018.JPEG  
  inflating: Car damage/level/validation/01-minor/0042.JPEG  
  inflating: Car damage/level/validation/01-minor/0048.JPEG  
  inflating: Car damage/level/validation/01-minor/0035.JPEG  
  inflating: Car damage/level/validation/01-minor/0014.JPEG  
  inflating: Car damage/level/validation/01-minor/0008.jpeg  
  inflating: Car damage/level/validation/01-minor/0013.JPEG  
  inflating: Car damage/level/validation/01-minor/0009.jpeg  
  inflating: Car damage/level/validation/01-minor/0037.JPEG  
  inflating: Car damage/level/validation/01-minor/0039.JPEG  
  inflating: Car damage/level/validation/01-min

#**Image Preprocessing**







**1. Import The ImageDataGenerator Library**

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

**2. Configure ImageDataGenerator Class**

In [6]:
#Creating augment.. on the training variable
train_datagen = ImageDataGenerator(rescale=1./255,shear_range = 0.1,zoom_range=0.1,horizontal_flip=True)

# Creating augment.. on the testing variable
test_datagen = ImageDataGenerator(rescale=1./255)

**3. Apply ImageDataGenerator Functionality To Trainset And Testset**

In [7]:
# Passing training data to train variable for body

xtrain = train_datagen.flow_from_directory('/content/drive/MyDrive/Colab Notebooks/Car damage/body/training',target_size=(224,224),class_mode='categorical',batch_size=10)

Found 979 images belonging to 3 classes.


In [8]:
# Passing testing data to test variable for body

xtest = test_datagen.flow_from_directory('/content/drive/MyDrive/Colab Notebooks/Car damage/body/validation',target_size=(224,224),class_mode='categorical',batch_size=10)

Found 171 images belonging to 3 classes.


In [9]:
# Passing training data to train variable for level

x_train = train_datagen.flow_from_directory('/content/drive/MyDrive/Colab Notebooks/Car damage/level/training',target_size=(224,224),class_mode='categorical',batch_size=10)

Found 979 images belonging to 3 classes.


In [10]:
# Passing testing data to test variable for level

x_test = test_datagen.flow_from_directory('/content/drive/MyDrive/Colab Notebooks/Car damage/level/validation',target_size=(224,224),class_mode='categorical',batch_size=10)

Found 171 images belonging to 3 classes.


# **Model Building for Body**

**1. Importing The Model Building Libraries**

In [11]:
#Import the library
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

**2. Loading The Model**

In [12]:
IMAGE_SIZE = [224, 224]

train_path = '/content/drive/MyDrive/Colab Notebooks/Car damage/body/training'
valid_path = '/content/drive/MyDrive/Colab Notebooks/Car damage/body/validation'

vgg16 = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)


**3. Adding Flatten Layer**

In [13]:
for layer in vgg16.layers:
    layer.trainable = False
folders = glob('/content/drive/MyDrive/Colab Notebooks/Car damage/body/training/*')

folders

['/content/drive/MyDrive/Colab Notebooks/Car damage/body/training/00-front',
 '/content/drive/MyDrive/Colab Notebooks/Car damage/body/training/02-side',
 '/content/drive/MyDrive/Colab Notebooks/Car damage/body/training/01-rear']

In [14]:
x = Flatten()(vgg16.output)
print('Length of Folders',len(folders))

Length of Folders 3


**4. Adding Output Layer**

In [15]:
prediction = Dense(len(folders), activation='softmax')(x)

**5. Creating A Model Object**

In [16]:
model = Model(inputs=vgg16.input, outputs=prediction)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

**6. Configure The Learning Process**

In [17]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

**7. Train The Model**

In [19]:
r = model.fit_generator(xtrain,validation_data=xtest,epochs=7,steps_per_epoch=len(xtrain),validation_steps=len(xtest))

Epoch 1/7


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """Entry point for launching an IPython kernel.


98/98 [==============================] - 422s 4s/step - loss: 0.6890 - accuracy: 0.7395 - val_loss: 0.8459 - val_accuracy: 0.6491
Epoch 2/7
98/98 [==============================] - 422s 4s/step - loss: 0.4548 - accuracy: 0.8233 - val_loss: 0.9873 - val_accuracy: 0.6199
Epoch 3/7
98/98 [==============================] - 414s 4s/step - loss: 0.3862 - accuracy: 0.8509 - val_loss: 1.0324 - val_accuracy: 0.6433
Epoch 4/7
98/98 [==============================] - 415s 4s/step - loss: 0.2771 - accuracy: 0.9060 - val_loss: 1.0637 - val_accuracy: 0.6550
Epoch 5/7
98/98 [==============================] - 414s 4s/step - loss: 0.2668 - accuracy: 0.9009 - val_loss: 1.2218 - val_accuracy: 0.6023
Epoch 6/7
98/98 [==============================] - 415s 4s/step - loss: 0.2418 - accuracy: 0.9152 - val_loss: 1.3472 - val_accuracy: 0.6491
Epoch 7/7
98/98 [==============================] - 414s 4s/step - loss: 0.1594 - accuracy: 0.9397 - val_loss: 1.0372 - val_accuracy: 0.6842


**8. Save The Model**

In [25]:
from tensorflow.keras.models import load_model
model.save('/content/drive/MyDrive/Colab Notebooks/Car damages/body.h5')

**9. Test The Model**

In [27]:
#import Required Lib
from tensorflow.keras.models import load_model
import cv2
from skimage.transform import resize
import numpy as np

In [26]:
model = load_model('/content/drive/MyDrive/Colab Notebooks/Car damages/body.h5')

In [28]:
def detect(frame):
  img = cv2.resize(frame,(224,224))
  img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

  if(np.max(img)>1):
    img = img/255.0
  img = np.array([img])
  prediction = model.predict(img)
  label = ["front","rear","side"]
  preds = label[np.argmax(prediction)]
  return preds

In [29]:
data = "/content/drive/MyDrive/Colab Notebooks/Car damage/body/training/00-front/0002.JPEG"
image = cv2.imread(data)
print(detect(image))

1/1 [==============================] - 1s 614ms/step
front


# **Model Building For Level**

**1. Importing The Model Building Lib**

In [30]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

**2. Loading The Model**

In [31]:
IMAGE_SIZE = [224, 224]

train_path = '/content/drive/MyDrive/Colab Notebooks/Car damage/level/training'
valid_path = '/content/drive/MyDrive/Colab Notebooks/Car damage/level/validation'
vgg16 = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

**3. Adding Flatten Layer**

In [32]:
for layer in vgg16.layers:
    layer.trainable = False
folders = glob('/content/drive/MyDrive/Colab Notebooks/Car damage/level/training/*')
folders

['/content/drive/MyDrive/Colab Notebooks/Car damage/level/training/02-moderate',
 '/content/drive/MyDrive/Colab Notebooks/Car damage/level/training/03-severe',
 '/content/drive/MyDrive/Colab Notebooks/Car damage/level/training/01-minor']

In [33]:
x = Flatten()(vgg16.output)
print("Length of Folders ",len(folders))

Length of Folders  3


**4. Adding Output Layer**

In [34]:
prediction = Dense(len(folders), activation='softmax')(x)

**5. Creating A Model Object**

In [35]:
model = Model(inputs=vgg16.input, outputs=prediction)
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

**6. Configure The Learning Process**

In [37]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

**7. Train The Model**

In [38]:
r = model.fit_generator(x_train,validation_data=x_test,epochs=4,steps_per_epoch=len(x_train),validation_steps=len(x_test))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """Entry point for launching an IPython kernel.


Epoch 1/4
98/98 [==============================] - 419s 4s/step - loss: 1.2847 - accuracy: 0.5424 - val_loss: 1.0048 - val_accuracy: 0.6257
Epoch 2/4
98/98 [==============================] - 414s 4s/step - loss: 0.7391 - accuracy: 0.7048 - val_loss: 0.8865 - val_accuracy: 0.6608
Epoch 3/4
98/98 [==============================] - 414s 4s/step - loss: 0.5511 - accuracy: 0.7732 - val_loss: 1.3959 - val_accuracy: 0.5322
Epoch 4/4
98/98 [==============================] - 415s 4s/step - loss: 0.4085 - accuracy: 0.8355 - val_loss: 1.1134 - val_accuracy: 0.6608


**8. Save The Model**

In [39]:
from tensorflow.keras.models import load_model

model.save('/content/drive/MyDrive/Colab Notebooks/Car damages/level.h5')

**9. Test The Model**

In [40]:
from tensorflow.keras.models import load_model
import cv2
from skimage.transform import resize
import numpy as np

In [41]:
model = load_model('/content/drive/MyDrive/Colab Notebooks/Car damages/level.h5')

In [42]:
def detect(frame):
  img = cv2.resize(frame,(224,224))
  img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

  if(np.max(img)>1):
    img = img/255.0
  img = np.array([img])
  prediction = model.predict(img)
  label = ["minor","moderate","severe"]
  preds = label[np.argmax(prediction)]
  return preds

In [45]:
data = "/content/drive/MyDrive/Colab Notebooks/Car damage/level/validation/01-minor/0005.JPEG"
image = cv2.imread(data)
print(detect(image))

1/1 [==============================] - 0s 447ms/step
minor
